In [12]:
import requests                         # for sending HTTP requests

from tqdm.notebook import tqdm, trange  # for progress bars

from scrapy import Selector             # for parsing HTML content
import pandas as pd

I ran into issues with 403 req, so trying headers

In [13]:
def fetch_page(url):
  """
  """
  
  try:
    response = requests.get(url, timeout=None) 
  except Exception as e:
    print('Error', e)
    pass
  if not response.ok:
    print("Something went wrong", response.status_code)
    pass
  html = response.content
  
  return Selector(text=html)

In [14]:
def extract_all_listings(page_sel): 
  """
  Inputs: 
    page_sel: a Selector object which contains 
      the contents of the Zillow website of houses
  Outputs:
    rental_list: a list of listings contained within page_sel
  """
  objects = page_sel.xpath('//div[contains(@class, "StyledPropertyCardDataWrapper")]')
  return objects
  #TODO 


In [35]:
def get_listing_info(listing_sel):
  """
  Inputs:
    listing_sel: a Selector object which contains the content
      of one listing
  Outputs:
    unit_dict: a dictionary which contains price per month, number of bedrooms, 
      number of bathrooms, sqft, postal code, and date listed
  """
  address = listing_sel.css("a ::text").get()
  zip_code = address[-5:]
  price = listing_sel.css("div span ::text").get()[1:]
  bbs = listing_sel.css("ul li b ::text").getall()
  bed = bbs[0]
  bath = bbs[1]
  sqft = bbs[2]
  print(bed, bath, sqft)
  print(address)
  print(price)
  print(zip_code)

In [36]:
sel = fetch_page('https://www.zillow.com/boston-ma/2_p/')
objects = sel.xpath('//div[contains(@class, "StyledPropertyCardDataWrapper")]')
print(objects[0])
get_listing_info(objects[0])

<div class="StyledPropertyCardDataWrapper-c11n-8-102-0__sc-hfbvv9-0 cXubMK property-card-data"><a href="https://www.zillow.com/homedetails/131-Brooks-St-East-Boston-MA-02128/59124037_zpid/" data-test="property-card-link" class="StyledPropertyCardDataArea-c11n-8-102-0__sc-10i1r6-0 klMkvj property-card-link" tabindex="0"><address data-test="property-card-addr">131 Brooks St, East Boston, MA 02128</address></a><div class="StyledPropertyCardDataArea-c11n-8-102-0__sc-10i1r6-0 eYtsWk">CENTURY 21 NORTH EAST, Pedro Cordano</div><div class="StyledPropertyCardDataArea-c11n-8-102-0__sc-10i1r6-0 gHLzem"><div class="PropertyCardWrapper__StyledPriceGridContainer-srp-8-102-0__sc-16e8gqd-0 eFBhnu"><span data-test="property-card-price" class="PropertyCardWrapper__StyledPriceLine-srp-8-102-0__sc-16e8gqd-1 vjmXt">$850,000</span></div></div><div class="StyledPropertyCardDataArea-c11n-8-102-0__sc-10i1r6-0 gAnZoM"><ul class="StyledPropertyCardHomeDetailsList-c11n-8-102-0__sc-1j0som5-0 exCsDV"><li><b>7</b> <